Sports clothing and athleisure attire is a huge industry, worth approximately [$193 billion in 2021](https://www.statista.com/statistics/254489/total-revenue-of-the-global-sports-apparel-market/) with a strong growth forecast over the next decade! 

In this notebook, you will undertake the role of a product analyst for an online sports clothing company. The company is specifically interested in how it can improve revenue. You will dive into product data such as pricing, reviews, descriptions, and ratings, as well as revenue and website traffic, to produce recommendations for its marketing and sales teams.  

You've been provided with five datasets to investigate:
* `info.csv`
* `finance.csv`
* `reviews.csv`
* `traffic.csv`
* `brands.csv`

The company has asked you to answer the following questions:

## What is the volume of products and average revenue for Adidas and Nike products based on price quartiles?

* Label products priced up to quartile one as `"Budget"`, quartile 2 as `"Average"`, quartile 3 as `"Expensive"`, and quartile 4 as `"Elite"`.
* Store as a `pandas` DataFrame called `adidas_vs_nike` containing the following columns: `"brand"`, `"price_label"`, `"num_products"`, and `"mean_revenue"`.

## Do any differences exist between the word count of a product's description and its mean rating?

* Store the results as a `pandas` DataFrame called `description_lengths` containing the following columns: `"description_length"`, `"mean_rating"`, `"num_reviews"`.

## How does the volume of products and median revenue vary between clothing and footwear?

* **Create** a `pandas` DataFrame called `product_types` containing the following columns: `"num_clothing_products"`, `"median_clothing_revenue"`, `"num_footwear_products"`, `"median_footwear_revenue"`.

# Reading Data

In [118]:
# Start coding here... 
import pandas as pd
data = ["info.csv" ,"finance.csv","reviews.csv","traffic.csv","brands.csv"]
for df in data : 
    name = df.split(".")[0]
    vars() ["df_"+name] = pd.read_csv(df)

In [119]:
df_info.head()

,product_name,product_id,description
0,NaN,AH2430,NaN
1,Women's adidas Originals Sleek Shoes,G27341,"A modern take on adidas sport heritage, tailor..."
2,Women's adidas Swim Puka Slippers,CM0081,These adidas Puka slippers for women's come wi...
3,Women's adidas Sport Inspired Questar Ride Shoes,B44832,"Inspired by modern tech runners, these women's..."
4,Women's adidas Originals Taekwondo Shoes,D98205,This design is inspired by vintage Taekwondo s...


In [120]:
df_finance.head()

,product_id,listing_price,sale_price,discount,revenue
0,AH2430,NaN,NaN,NaN,NaN
1,G27341,75.99,37.99,0.5,1641.17
2,CM0081,9.99,5.99,0.4,398.93
3,B44832,69.99,34.99,0.5,2204.37
4,D98205,79.99,39.99,0.5,5182.70


In [121]:
df_reviews.head()

,product_id,rating,reviews
0,AH2430,NaN,NaN
1,G27341,3.3,24.0
2,CM0081,2.6,37.0
3,B44832,4.1,35.0
4,D98205,3.5,72.0


In [122]:
df_traffic.head()

,product_id,last_visited
0,AH2430,2018-05-19 15:13:00
1,G27341,2018-11-29 16:16:00
2,CM0081,2018-02-01 10:27:00
3,B44832,2018-09-07 20:06:00
4,D98205,2019-07-18 15:26:00


In [123]:
df_brands.head()

,product_id,brand
0,AH2430,NaN
1,G27341,Adidas
2,CM0081,Adidas
3,B44832,Adidas
4,D98205,Adidas


### Merging data

In [124]:
df1 = df_info.merge(df_finance,on="product_id")
df2 = df_reviews.merge(df_traffic,on="product_id")
df3 = df1.merge(df_brands,on="product_id")
df = df2.merge(df3,on="product_id")

In [125]:
df.head()

,product_id,rating,reviews,last_visited,product_name,description,listing_price,sale_price,discount,revenue,brand
0,AH2430,NaN,NaN,2018-05-19 15:13:00,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,G27341,3.3,24.0,2018-11-29 16:16:00,Women's adidas Originals Sleek Shoes,"A modern take on adidas sport heritage, tailor...",75.99,37.99,0.5,1641.17,Adidas
2,CM0081,2.6,37.0,2018-02-01 10:27:00,Women's adidas Swim Puka Slippers,These adidas Puka slippers for women's come wi...,9.99,5.99,0.4,398.93,Adidas
3,B44832,4.1,35.0,2018-09-07 20:06:00,Women's adidas Sport Inspired Questar Ride Shoes,"Inspired by modern tech runners, these women's...",69.99,34.99,0.5,2204.37,Adidas
4,D98205,3.5,72.0,2019-07-18 15:26:00,Women's adidas Originals Taekwondo Shoes,This design is inspired by vintage Taekwondo s...,79.99,39.99,0.5,5182.70,Adidas


## Removing Null Values

In [126]:
df.dropna(inplace=True)

In [127]:
df.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 2873 entries, 1 to 3178
Data columns (total 11 columns):
 #   Column         Non-Null Count  Dtype  
---  ------         --------------  -----  
 0   product_id     2873 non-null   object 
 1   rating         2873 non-null   float64
 2   reviews        2873 non-null   float64
 3   last_visited   2873 non-null   object 
 4   product_name   2873 non-null   object 
 5   description    2873 non-null   object 
 6   listing_price  2873 non-null   float64
 7   sale_price     2873 non-null   float64
 8   discount       2873 non-null   float64
 9   revenue        2873 non-null   float64
 10  brand          2873 non-null   object 
dtypes: float64(6), object(5)
memory usage: 269.3+ KB


## Sales performance of Adidas and Nike products

In [128]:
#it is better to use to qcut() rather than cut() because most of the time data is not equally distributed but rather skewed  
labels = ["Budget","Average","Expensive","Elite"]
df["price_label"] = pd.qcut(df["listing_price"],q=4,labels=labels)

In [129]:
df["price_label"].head()

1    Expensive
2       Budget
3    Expensive
4    Expensive
5      Average
Name: price_label, dtype: category
Categories (4, object): ['Budget' < 'Average' < 'Expensive' < 'Elite']

In [130]:
adidas_vs_nike = df.groupby(["brand","price_label"]) \
    .agg(num_products = ("product_id","count"),mean_revenue=("revenue","mean"))\
    .round(2)\
    .reset_index(drop=False)
adidas_vs_nike

,brand,price_label,num_products,mean_revenue
0,Adidas,Budget,538,2050.97
1,Adidas,Average,599,2982.30
2,Adidas,Expensive,707,4599.58
3,Adidas,Elite,533,8424.18
4,Nike,Budget,321,1664.33
5,Nike,Average,8,675.59
6,Nike,Expensive,43,472.74
7,Nike,Elite,124,1418.42


## Relationship between product description lengths, ratings, and reviews

In [131]:
df["description_length"] = df["description"].str.len()
#largest decscription length
max(df["description"].str.len())

687

In [132]:
# Upper description length limits
lengths = [0, 100, 200, 300, 400, 500, 600, 700]

# Description length labels
labels = ["100", "200", "300", "400", "500", "600", "700"]

In [133]:
df["description_length"] = pd.cut(df["description_length"],bins=lengths,labels=labels)

In [134]:
description_lengths = df.groupby("description_length",as_index=False)\
.agg(mean_rating =("rating","mean"),num_reviews  = ("reviews","count")).round(2)
                                 
description_lengths

,description_length,mean_rating,num_reviews
0,100,2.26,7
1,200,3.17,483
2,300,3.29,1642
3,400,3.31,601
4,500,3.41,110
5,600,3.12,15
6,700,3.65,15


## Comparing footwear and clothing products

In [135]:
footwear_list = "shoe*|trainer*|foot*"
footewear = df[df["description"].str.contains(footwear_list)]
clothing = df[~df["description"].str.contains(footwear_list)]
clothing.dropna(inplace=True)

In [136]:
product_types = pd.DataFrame({
                "num_clothing_products" : len(clothing),
                "median_clothing_revenue" : clothing["revenue"].median(),
                "num_footwear_products" : len(footewear),
                "median_footwear_revenue" : footewear["revenue"].median(),},
                index = [0])
product_types

,num_clothing_products,median_clothing_revenue,num_footwear_products,median_footwear_revenue
0,439,683.73,2434,3073.3
